In [11]:
import pandas as pd
import numpy as np
import operator
import re
import datetime 
from datetime import timedelta
from collections import OrderedDict
import seaborn as sns
import pickle
import os.path
from matplotlib.colors import ListedColormap
import matplotlib.colors as colors
import matplotlib.cm as cmx
import matplotlib.pyplot as plt
#from mpl_toolkits.basemap import Basemap
#from geopy.geocoders import Nominatim
import math
import warnings
warnings.filterwarnings('ignore')
from timezonefinder import TimezoneFinder
tf = TimezoneFinder()
import pytz

top_airtpots= ['ATL', 'DEN', 'DFW', 'IAH', 'LAS', 'LAX', 'ORD', 'PHX', 'SEA', 'SFO']
len(top_airtpots)

10

### 1. Reading the Airports file
There 322 airports in total. I used only top 10 most busiest (in terms of traffic) airports 

In [12]:
airports=pd.read_csv('airports.csv')
print('There are {} airports'.format (airports.shape[0]))

There are 322 airports


Filtering only the airports that are needed.

In [13]:
airports= airports[airports.IATA_CODE.isin(top_airtpots)].reset_index(inplace=False)

In [14]:
print('There are {} airports'.format(airports.shape[0]))

There are 10 airports


### 2. Getting the timezone for each airports
This data will be used to get weather data, details will be discussed then

### 2.1 Example Timezone Finding 


In [15]:
newtimezone= tf.closest_timezone_at(lng=-84.42694, lat=33.64044)
print('Time zone for Hartsfield-Jackson Atlanta International Airport is', newtimezone)

Time zone for Hartsfield-Jackson Atlanta International Airport is America/Chicago


### 2.2 Example Finding Local Time at New Timezone
This will be important when getting accurate weather data.


In [16]:
epoch2015_local= 1420092000 #Thursday, January 1, 2015 12:00:00 AM
print('My timestamp on Thursday, January 1, 2015 12:00:00 AM  : ',epoch2015_local)
local_time =datetime.datetime.fromtimestamp(epoch2015_local)
timezone = pytz.timezone('America/New_York')
d_aware = timezone.localize(local_time)
print('Timestamp at the desired timezone at the same time     : ',d_aware.timestamp())

My timestamp on Thursday, January 1, 2015 12:00:00 AM  :  1420092000
Timestamp at the desired timezone at the same time     :  1420088400.0


### 3. MakeTimeZone Function
Returns the local time in comparison to my local time on Jan 1,2015 at 00:00:00 am

In [17]:
def maketimezone(LONGITUDE,LATITUDE):
    
    # Determining the timezone
    newtimezone= tf.closest_timezone_at(lng=LONGITUDE, lat=LATITUDE)

    return newtimezone

In [18]:
def maketimestamp(LONGITUDE,LATITUDE):
    
    # Determining the timezone
    newtimezone= tf.closest_timezone_at(lng=LONGITUDE, lat=LATITUDE)
    
    epoch2015_local= 1420092000 #Thursday, January 1, 2015 12:00:00 AM
    
    local_time =datetime.datetime.fromtimestamp(epoch2015_local)
    
    timezone = pytz.timezone(newtimezone)
    
    newtimestamp = timezone.localize(local_time)
    
    return newtimestamp.timestamp()

In [19]:
airports['Timezone']=''
for i in range(len(airports)):
    airports['Timezone'][i]= maketimezone(airports.LONGITUDE.iloc[i],airports.LATITUDE.iloc[i])

In [20]:
airports['Timestamp']=''
for i in range(len(airports)):
    airports['Timestamp'][i]= maketimestamp(airports.LONGITUDE.iloc[i],airports.LATITUDE.iloc[i])

In [21]:
airports['Timezone_Diff']=(1420092000-airports.Timestamp)/3600

### 4. Save Changes

In [22]:
airports.to_csv('Capstone 1 - Airports.csv')

In [23]:
airports

,index,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE,Timezone,Timestamp,Timezone_Diff
0,20,ATL,Hartsfield-Jackson Atlanta International Airport,Atlanta,GA,USA,33.64044,-84.42694,America/Chicago,1.42009e+09,0
1,85,DEN,Denver International Airport,Denver,CO,USA,39.85841,-104.66700,America/Denver,1.4201e+09,-1
2,86,DFW,Dallas/Fort Worth International Airport,Dallas-Fort Worth,TX,USA,32.89595,-97.03720,America/Chicago,1.42009e+09,0
3,151,IAH,George Bush Intercontinental Airport,Houston,TX,USA,29.98047,-95.33972,America/Chicago,1.42009e+09,0
4,174,LAS,McCarran International Airport,Las Vegas,NV,USA,36.08036,-115.15233,America/Los_Angeles,1.4201e+09,-2
5,176,LAX,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,America/Los_Angeles,1.4201e+09,-2
6,228,ORD,Chicago O'Hare International Airport,Chicago,IL,USA,41.97960,-87.90446,America/Chicago,1.42009e+09,0
7,239,PHX,Phoenix Sky Harbor International Airport,Phoenix,AZ,USA,33.43417,-112.00806,America/Phoenix,1.4201e+09,-1
8,277,SEA,Seattle-Tacoma International Airport,Seattle,WA,USA,47.44898,-122.30931,America/Los_Angeles,1.4201e+09,-2
9,278,SFO,San Francisco International Airport,San Francisco,CA,USA,37.61900,-122.37484,America/Los_Angeles,1.4201e+09,-2
